# spam or ham classifier

In [1]:
import pandas as pd
import nltk
import string
import re
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kuwar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kuwar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# reading all the files and renaming

In [2]:
mssg=pd.read_csv('spam.csv', encoding='latin1')
mssg.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
mssg.rename(columns={'v1':'classification','v2':'message'},inplace=True)
mssg.head()
mssg['classification'].dtype
mssg['message'].dtype

dtype('O')

# preproccesing

In [3]:

for i in range(0, len(mssg)):
  review = re.sub('[^a-zA-Z]', ' ', mssg['message'][i])
  review = review.lower()
  review = review.split()
    
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
#corpus

# training using naive bayes

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

y=pd.get_dummies(mssg['classification'])
y=y.iloc[:,1].values
#import pickle ## importing pickle used for dumping models
#pickle.dump(cv, open('cv.pkl', 'wb')) ## saving to into cv.pkl file

# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

y_pred

print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))



import pickle
pickle.dump(spam_detect_model, open("spam.pkl", "wb"))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       949
           1       0.96      0.95      0.96       166

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115

[[943   6]
 [  8 158]]


# training using random forest classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import confusion_matrix, classification_report 
rfc=RandomForestClassifier(n_estimators=200)
rfc.fit(X_train,y_train)
predict_rfc=rfc.predict(X_test)
print(classification_report(y_test,predict_rfc))
print(confusion_matrix(y_test,predict_rfc))
print(predict_rfc)
len(predict_rfc)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       949
           1       0.99      0.86      0.92       166

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115

[[948   1]
 [ 23 143]]
[0 0 0 ... 0 0 0]


1115

# load the model

In [13]:
loaded_model = pickle.load(open("spam.pkl", "rb"))
loaded_model.predict(X_test)
loaded_model.score(X_test,y_test)


0.9874439461883409

# checking the model by passing the input

In [14]:
def new_review(rev):
  rev = rev
  rev = re.sub('[^a-zA-Z]', ' ',rev)
  rev = rev.lower()
  rev = rev.split()
    
  rev = [ps.stem(word) for word in rev if not word in stopwords.words('english')]
  rev = ' '.join(rev)
  new_corpus=[rev]
  new_X_test = cv.transform(new_corpus).toarray()
  new_y_pred = loaded_model.predict(new_X_test)
  return new_y_pred
new_review = new_review(str(input("Enter new review...")))
if new_review[0]==1:
  print("SPAM")
else :
  print("NOT SPAM")

Enter new review...wanna fuck me
NOT SPAM
